
# Data Preparation

In [2]:
import pandas as pd
import numpy as np

In [10]:
# Load the dataset
df = pd.read_csv('dataset/Crops_District_Production.csv')
# basic Information
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nColumn info:")
print(df.info())
# Check for any missing values
print("\nMissing values:")
print(df.isnull().sum())

Dataset shape: (11002, 6)

First few rows:
         date  state    district   crop_type  crop_species  production
0  2017-01-01  Johor  Batu Pahat  cash_crops       cassava       920.5
1  2017-01-01  Johor  Batu Pahat  cash_crops    groundnuts         0.0
2  2017-01-01  Johor  Batu Pahat  cash_crops    sweet_corn         0.0
3  2017-01-01  Johor  Batu Pahat  cash_crops  sweet_potato       350.0
4  2017-01-01  Johor  Batu Pahat  cash_crops           yam       395.4

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11002 entries, 0 to 11001
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          11002 non-null  object 
 1   state         11002 non-null  object 
 2   district      11002 non-null  object 
 3   crop_type     11002 non-null  object 
 4   crop_species  11002 non-null  object 
 5   production    11002 non-null  float64
dtypes: float64(1), object(5)
memory usage: 515.8+ KB
None

Missi

In [11]:
# Group by state, district, crop_type and find top 5 species by production
def transform_to_top5_format(df):
    """
    Transform crop data to show top 5 species per district/crop_type
    Modern pandas approach that avoids all warnings
    """
    # Sort by production in descending order
    df_sorted = df.sort_values(['state', 'district', 'crop_type', 'production'], 
                              ascending=[True, True, True, False])
    
    # Get top 5 for each group and explicitly create a copy
    top5_df = df_sorted.groupby(['state', 'district', 'crop_type']).head(5).copy()
    
    # Create ranking within each group using .loc to avoid warning
    top5_df.loc[:, 'rank'] = top5_df.groupby(['state', 'district', 'crop_type']).cumcount() + 1
    
    # Pivot to get the desired format
    result = top5_df.pivot_table(
        index=['state', 'district', 'crop_type'],
        columns='rank',
        values='crop_species',
        aggfunc='first'
    ).reset_index()
    
    # Rename columns to crop_rank_1, crop_rank_2, etc.
    result.columns.name = None  # Remove the 'rank' column name
    new_columns = ['state', 'district', 'crop_type'] + [f'crop_rank_{i}' for i in range(1, 6)]
    result.columns = new_columns[:len(result.columns)]
    
    return result

# Transform the data
df = transform_to_top5_format(df)

# Display results
print("Transformed dataset shape:", df.shape)
print("\nFirst 10 rows:")
print(df.head(10))

Transformed dataset shape: (903, 8)

First 10 rows:
   state     district         crop_type           crop_rank_1  \
0  Johor   Batu Pahat        cash_crops               cassava   
1  Johor   Batu Pahat             fruit             pineapple   
2  Johor   Batu Pahat             herbs  fragrant_lemon_grass   
3  Johor   Batu Pahat  industrial_crops               coconut   
4  Johor   Batu Pahat            spices       calamondin_lime   
5  Johor   Batu Pahat         vegetable             long_bean   
6  Johor  Johor Bahru        cash_crops               cassava   
7  Johor  Johor Bahru             fruit                banana   
8  Johor  Johor Bahru             herbs  fragrant_lemon_grass   
9  Johor  Johor Bahru  industrial_crops               coconut   

         crop_rank_2       crop_rank_3       crop_rank_4   crop_rank_5  
0                yam      sweet_potato        groundnuts    sweet_corn  
1             papaya            durian            banana         guava  
2            

In [12]:
# Combine all csv data into single dataset:
print("Load all CSV files...")

# disctrict data and corps
print(f"District data and Corps shape: {df.shape}")

# Average Yearly Weather Information
df_weather = pd.read_csv('dataset/district_with_weather_data.csv')
print(f"Weather information shape: {df_weather.shape}")

Load all CSV files...
District data and Corps shape: (903, 8)
Weather information shape: (155, 14)


in here, we see that the disctrict row data is bigger than the climate and elevation data because of multiple crop_type on each district.

In [14]:
# data cleaning
# Clean column names (remove extra spaces)
df_weather.columns = df_weather.columns.str.strip()

# Standardize state and district names (remove extra spaces, standardize case)
for dfw in [df_weather]:
    dfw['state'] = dfw['state'].str.strip().str.title()
    dfw['district'] = dfw['district'].str.strip().str.title()

print("Data cleaned - standardized state/district names")

Data cleaned - standardized state/district names


In [16]:
# Merging dataset
print("MERGING DATASETS")

# Step 1: Start with main crop production data (using existing df)
combined_df = df.copy()
print(f"Starting with crop data: {combined_df.shape}")

# Step 2: Merge with weather data
combined_df = pd.merge(
    combined_df, 
    df_weather, 
    on=['state', 'district'], 
    how='left'
)
print(f"After adding climate data: {combined_df.shape}")


MERGING DATASETS
Starting with crop data: (903, 8)
After adding climate data: (903, 20)


check the combined data

In [17]:
print("head")
combined_df.head()

head


,state,district,crop_type,crop_rank_1,crop_rank_2,crop_rank_3,crop_rank_4,crop_rank_5,lat,lon,uv_index_max,temperature_2m_mean,cloud_cover_mean,relative_humidity_2m_mean,sunshine_duration,precipitation_sum,et0_fao_evapotranspiration,temperature_2m_min,temperature_2m_max,shortwave_radiation_sum
0,Johor,Batu Pahat,cash_crops,cassava,yam,sweet_potato,groundnuts,sweet_corn,1.847258,102.93467,7.83,27.55,90.33,84.63,9420495.95,2920.6,1250.01,24.88,30.98,5847.1
1,Johor,Batu Pahat,fruit,pineapple,papaya,durian,banana,guava,1.847258,102.93467,7.83,27.55,90.33,84.63,9420495.95,2920.6,1250.01,24.88,30.98,5847.1
2,Johor,Batu Pahat,herbs,fragrant_lemon_grass,cekur,lemon_myrtle,aloe_vera,basil,1.847258,102.93467,7.83,27.55,90.33,84.63,9420495.95,2920.6,1250.01,24.88,30.98,5847.1
3,Johor,Batu Pahat,industrial_crops,coconut,coffee,roselle,mushroom,areca_nut,1.847258,102.93467,7.83,27.55,90.33,84.63,9420495.95,2920.6,1250.01,24.88,30.98,5847.1
4,Johor,Batu Pahat,spices,calamondin_lime,lemon_grass,greater_galangal,pink_cone_ginger,lime,1.847258,102.93467,7.83,27.55,90.33,84.63,9420495.95,2920.6,1250.01,24.88,30.98,5847.1


In [18]:
print("random")
combined_df.iloc[6:10]

random


,state,district,crop_type,crop_rank_1,crop_rank_2,crop_rank_3,crop_rank_4,crop_rank_5,lat,lon,uv_index_max,temperature_2m_mean,cloud_cover_mean,relative_humidity_2m_mean,sunshine_duration,precipitation_sum,et0_fao_evapotranspiration,temperature_2m_min,temperature_2m_max,shortwave_radiation_sum
6,Johor,Johor Bahru,cash_crops,cassava,yellow_sugar_cane,sweet_potato,yam,sweet_corn,1.458199,103.764906,7.86,28.25,88.09,81.01,10384534.3,2219.4,1316.78,25.98,31.52,5913.82
7,Johor,Johor Bahru,fruit,banana,pineapple,durian,rambutan,papaya,1.458199,103.764906,7.86,28.25,88.09,81.01,10384534.3,2219.4,1316.78,25.98,31.52,5913.82
8,Johor,Johor Bahru,herbs,fragrant_lemon_grass,aloe_vera,basil,belalai_gajah,betel_vine,1.458199,103.764906,7.86,28.25,88.09,81.01,10384534.3,2219.4,1316.78,25.98,31.52,5913.82
9,Johor,Johor Bahru,industrial_crops,coconut,mushroom,areca_nut,coffee,honey_bee,1.458199,103.764906,7.86,28.25,88.09,81.01,10384534.3,2219.4,1316.78,25.98,31.52,5913.82


In [19]:
print("tail")
combined_df.tail()

tail


,state,district,crop_type,crop_rank_1,crop_rank_2,crop_rank_3,crop_rank_4,crop_rank_5,lat,lon,uv_index_max,temperature_2m_mean,cloud_cover_mean,relative_humidity_2m_mean,sunshine_duration,precipitation_sum,et0_fao_evapotranspiration,temperature_2m_min,temperature_2m_max,shortwave_radiation_sum
898,W.P. Labuan,W.P. Labuan,cash_crops,cassava,yellow_sugar_cane,sweet_corn,NaN,NaN,5.28704,115.26461,8.03,28.14,83.56,82.72,12098522.28,2089.6,1523.96,26.07,31.62,7198.07
899,W.P. Labuan,W.P. Labuan,fruit,banana,jackfruit,mango,rambutan,cempedak,5.28704,115.26461,8.03,28.14,83.56,82.72,12098522.28,2089.6,1523.96,26.07,31.62,7198.07
900,W.P. Labuan,W.P. Labuan,industrial_crops,coconut,NaN,NaN,NaN,NaN,5.28704,115.26461,8.03,28.14,83.56,82.72,12098522.28,2089.6,1523.96,26.07,31.62,7198.07
901,W.P. Labuan,W.P. Labuan,spices,calamondin_lime,lemon_grass,NaN,NaN,NaN,5.28704,115.26461,8.03,28.14,83.56,82.72,12098522.28,2089.6,1523.96,26.07,31.62,7198.07
902,W.P. Labuan,W.P. Labuan,vegetable,mustard,water_spinach,chinese_spinach,sweet_shoot,cucumber,5.28704,115.26461,8.03,28.14,83.56,82.72,12098522.28,2089.6,1523.96,26.07,31.62,7198.07


In [20]:
print("FINAL COMBINED DATASET")
print(f"Final dataset shape: {combined_df.shape}")
print(f"Columns: {combined_df.columns}")

print("\nMissing values in combined dataset:")
missing_values = combined_df.isnull().sum()
missing_values = missing_values[missing_values > 0]
if len(missing_values) > 0:
    print(missing_values)
else:
    print("No missing values!")

FINAL COMBINED DATASET
Final dataset shape: (903, 20)
Columns: Index(['state', 'district', 'crop_type', 'crop_rank_1', 'crop_rank_2',
       'crop_rank_3', 'crop_rank_4', 'crop_rank_5', 'lat', 'lon',
       'uv_index_max', 'temperature_2m_mean', 'cloud_cover_mean',
       'relative_humidity_2m_mean', 'sunshine_duration', 'precipitation_sum',
       'et0_fao_evapotranspiration', 'temperature_2m_min',
       'temperature_2m_max', 'shortwave_radiation_sum'],
      dtype='object')

Missing values in combined dataset:
crop_rank_2     41
crop_rank_3     50
crop_rank_4    111
crop_rank_5    189
dtype: int64


In [45]:
# Since there are many missing values in avg_rainfall_mm and avg_temp_c, we will use the median of their state to fill in the missing values.
combined_df['avg_rainfall_mm'] = combined_df.groupby('state')['avg_rainfall_mm'].transform(lambda x: x.fillna(x.median()))
combined_df['avg_temp_c'] = combined_df.groupby('state')['avg_temp_c'].transform(lambda x: x.fillna(x.median()))


print("\nMissing values in combined dataset:")
missing_values = combined_df.isnull().sum()
missing_values = missing_values[missing_values > 0]
if len(missing_values) > 0:
    print(missing_values)
else:
    print("No missing values!")


# Check which specific state-district combinations are missing climate data
missing_climate = combined_df[combined_df['avg_rainfall_mm'].isnull()]
print("State-District combinations missing climate data:")
print(missing_climate[['state', 'district']].drop_duplicates())
print(f"Number of unique combinations missing: {missing_climate[['state', 'district']].drop_duplicates().shape[0]}")


Missing values in combined dataset:
avg_rainfall_mm    26
avg_temp_c         26
dtype: int64
State-District combinations missing climate data:
            state                district
32         Labuan             W.P. Labuan
54          Perak           Batang Padang
55          Perak           Muallim (New)
56          Perak             Hilir Perak
57          Perak       Bagan Datuk (New)
58          Perak              Hulu Perak
59          Perak                  Kampar
60          Perak                  Kerian
61          Perak                   Kinta
62          Perak           Kuala Kangsar
63          Perak        Larut Dan Matang
64          Perak                 Manjung
65          Perak            Perak Tengah
67   Pulau Pinang              Barat Daya
68   Pulau Pinang  Seberang Perai Selatan
69   Pulau Pinang   Seberang Perai Tengah
70   Pulau Pinang    Seberang Perai Utara
71   Pulau Pinang              Timur Laut
147     Trengganu                   Besut
148     Trenggan